In [1]:
!pip install llama-index
!pip install llama-index-embeddings-huggingface
!pip install accelerate
!pip install auto-gptq
!pip install optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 43.6 MB/s eta 0:00:00
  Using cached nvidia_cu

In [2]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from llama_index.core import Document

In [3]:
import os
import xml.etree.ElementTree as ET

class XMLParentMap:
    def __init__(self, tree):
        self.tree = tree
        self.parent_map = self.build_parent_map()

    def build_parent_map(self):
        parent_map = {}
        for p in self.tree.iter():
            for c in p:
                if c in parent_map:
                    parent_map[c].append(p)
                else:
                    parent_map[c] = [p]
        return parent_map

    def get_all_parents(self, element):
        all_parents = []
        self._collect_all_parents(element, all_parents)
        return all_parents

    def _collect_all_parents(self, element, all_parents):
        if element in self.parent_map:
            parents = self.parent_map[element]
            for parent in parents:
                all_parents.append(parent)
                self._collect_all_parents(parent, all_parents)

In [4]:
class XMLDirectoryReader:
    def __init__(self, directory):
        self.directory = directory

    def load_data(self):
        documents = []
        for filename in os.listdir(self.directory):
            if filename.endswith(".xml"):
                file_path = os.path.join(self.directory, filename)
                with open(file_path, 'r', encoding='utf-8') as file:
                    tree = ET.parse(file)
                    root = tree.getroot()
                    parent_map_instance = XMLParentMap(tree)
                    text_content = self.extract_text_from_xml(root, parent_map_instance)
                    if text_content:  # Vérifie que le texte n'est pas vide
                        documents.append(text_content)
        return documents

    def extract_text_from_xml(self, root, parent_map):
        namespaces = {'tei': 'http://www.tei-c.org/ns/1.0'}
        texts = []
        for elem in root.iter():
            all_parents = parent_map.get_all_parents(elem)

            # Vérifier si l'élément ou un de ses parents est dans <biblStruct>
            if any(p.tag in ['{http://www.tei-c.org/ns/1.0}biblStruct', '{http://www.tei-c.org/ns/1.0}figure'] for p in all_parents):
                continue  # Passer à l'élément suivant si dans <biblStruct>


            # Exclure certains éléments spécifiques (comme les métadonnées ou les informations de publication)
            if elem.tag in ['{http://www.tei-c.org/ns/1.0}title', '{http://www.tei-c.org/ns/1.0}head', '{http://www.tei-c.org/ns/1.0}p']:
                if elem.text and elem.text.strip():  # Exclure les éléments avec du texte vide
                    text = elem.text.strip()
                    if elem.tag == '{http://www.tei-c.org/ns/1.0}title':
                        text = f"\n\ntitle : {text}"  # Ajouter le préfixe "title :"
                    elif elem.tag == '{http://www.tei-c.org/ns/1.0}head':
                        text = f"\nhead : {text}"  # Ajouter le préfixe "head :"
                    texts.append(text)
        return "\n".join(texts)

In [5]:
# import any embedding model on HF hub (https://huggingface.co/spaces/mteb/leaderboard)
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
# Settings.embed_model = HuggingFaceEmbedding(model_name="thenlper/gte-large") # alternative model

Settings.llm = None
Settings.chunk_size = 128
Settings.chunk_overlap = 25

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

LLM is explicitly disabled. Using MockLLM.


In [7]:
documents = XMLDirectoryReader("data/Medical_Guidelines/XML").load_data()

In [8]:
print(documents[0])



title : ESCMID/EUCIC clinical practice guidelines on perioperative antibiotic prophylaxis in patients colonized by multidrug-resistant Gram-negative bacteria before surgery
Copyright Elsevier BV
a b s t r a c t Scope: The aim of the guidelines is to provide recommendations on perioperative antibiotic prophylaxis (PAP) in adult inpatients who are carriers of multidrug-resistant Gram-negative bacteria (MDR-GNB) before surgery. Methods: These evidence-based guidelines were developed after a systematic review of published studies on PAP targeting the following MDR-GNB: extended-spectrum cephalosporin-resistant Enterobacterales, carbapenem-resistant Enterobacterales (CRE), aminoglycoside-resistant Enterobacterales, fluoroquinolone-resistant Enterobacterales, cotrimoxazole-resistant Stenotrophomonas maltophilia, carbapenem-resistant Acinetobacter baumannii (CRAB), extremely drug-resistant Pseudomonas aeruginosa, colistin-resistant Gram-negative bacteria, and pan-drug-resistant Gram-negativ

In [9]:
# Transformer les chaînes de texte en objets Document
i = 1
document_objects = [Document(text=doc, title=f"Document {i+1}") for i, doc in enumerate(documents)]

In [10]:
print(document_objects[0])

Doc ID: 9092f193-b34f-44d6-b148-89c649b92316
Text: title : ESCMID/EUCIC clinical practice guidelines on
perioperative antibiotic prophylaxis in patients colonized by
multidrug-resistant Gram-negative bacteria before surgery Copyright
Elsevier BV a b s t r a c t Scope: The aim of the guidelines is to
provide recommendations on perioperative antibiotic prophylaxis (PAP)
in adult inpatients who are...


In [11]:
# store docs into vector DB
index = VectorStoreIndex.from_documents(document_objects)

In [12]:
# set number of docs to retreive
top_k = 3

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=top_k,
)

In [13]:
# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)

In [14]:
# query documents
query = "What is MDR-GNB??"
response = query_engine.query(query)

In [15]:
# reformat response
context = "Context:\n"
for i in range(top_k):
    context = context + response.source_nodes[i].text + "\n\n"

print(context)

Context:
To address the benefits of presurgical screening for MDR-GNB to inform targeted PAP in carriers before surgery, the articles reporting the rates of postoperative infections in MDR-GNB carriers vs. noncarriers were reviewed.
Sampling techniques and microbiological practices were not reviewed or discussed because they were beyond the scope of these guidelines.
The main research questions addressed by the guidelines include:
1. Should screening for MDR-GNB be recommended before surgery and when? 2. Which PAP have been evaluated for patients colonized with the target MDR-GNB? 3.

Please see Supplementary Material for details on the guideline development.

head : Literature search and data extraction
A systematic review of the published literature was performed, including studies evaluating PAP in adult inpatients (aged 18 years) colonized with MDR-GNB before surgery.

(e) extremely drug-resistant Pseudomonas aeruginosa (XDRPA), (f) cotrimoxazole-resistant Stenotrophomonas maltophi

In [16]:
# load model
model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4481: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.la

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [17]:
# prompt (no context)
intstructions_string = f"""MedGPT, functioning as a virtual data science consultant on clinical questions, communicates in clear, accessible language, escalating to technical depth upon request. \
It reacts to questions aptly and ends responses with its signature '–MedGPT'. \
MedGPT will tailor the length of its responses to match the client's question, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging.

Please respond to the following question.
"""
prompt_template = lambda question: f'''[INST] {intstructions_string} \n{question} \n[/INST]'''

In [18]:
question = "What is MDR-GNB?"

prompt = prompt_template(question)
print(prompt)

[INST] MedGPT, functioning as a virtual data science consultant on clinical questions, communicates in clear, accessible language, escalating to technical depth upon request. It reacts to questions aptly and ends responses with its signature '–MedGPT'. MedGPT will tailor the length of its responses to match the client's question, providing concise acknowledgments to brief expressions of gratitude or feedback, thus keeping the interaction natural and engaging.

Please respond to the following question.
 
What is MDR-GNB? 
[/INST]


In [20]:
model.eval()

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] MedGPT, functioning as a virtual data science consultant on clinical questions, communicates in clear, accessible language, escalating to technical depth upon request. It reacts to questions aptly and ends responses with its signature '–MedGPT'. MedGPT will tailor the length of its responses to match the client's question, providing concise acknowledgments to brief expressions of gratitude or feedback, thus keeping the interaction natural and engaging.

Please respond to the following question.
 
What is MDR-GNB? 
[/INST] MDR-GNB stands for Multidrug-Resistant Gram-Negative bacteria. These are types of bacteria that have developed resistance to multiple antibiotics, making infections caused by them harder to treat. Gram-negative bacteria have a unique cell wall structure that makes it more difficult for antibiotics to penetrate and kill the bacteria compared to Gram-positive bacteria. The multidrug resistance in MDR-GNB makes these infections a significant public health conc

In [21]:
# prompt (with context)
prompt_template_w_context = lambda context, question: f"""[INST]MedGPT, functioning as a virtual data science consultant on clinical questions, communicates in clear, accessible language, escalating to technical depth upon request. \
It reacts to questions aptly and ends responses with its signature '–MedGPT'. \
MedGPT will tailor the length of its responses to match the client's question, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging.

{context}
Please respond to the following question. Use the context above if it is helpful.

{question}
[/INST]
"""

In [22]:
prompt = prompt_template_w_context(context, question)

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST]MedGPT, functioning as a virtual data science consultant on clinical questions, communicates in clear, accessible language, escalating to technical depth upon request. It reacts to questions aptly and ends responses with its signature '–MedGPT'. MedGPT will tailor the length of its responses to match the client's question, providing concise acknowledgments to brief expressions of gratitude or feedback, thus keeping the interaction natural and engaging.

Context:
To address the benefits of presurgical screening for MDR-GNB to inform targeted PAP in carriers before surgery, the articles reporting the rates of postoperative infections in MDR-GNB carriers vs. noncarriers were reviewed.
Sampling techniques and microbiological practices were not reviewed or discussed because they were beyond the scope of these guidelines.
The main research questions addressed by the guidelines include:
1. Should screening for MDR-GNB be recommended before surgery and when? 2. Which PAP have been ev